# 注意力分数

In [1]:
import math
import torch
from torch import nn
from d2l import torch as d2l

首先我们需要实现一个`sequence_mask`函数，用于将无效的、用于填充的（padding）部分用一个特定的值覆盖掉。这在之后的transformer模型用也经常使用。  
这个函数的基本思想是，传入一个`tensor X`（2D tensor，这里要注意，一般我们的tensor是3D，包括`batch_size`,`key_num` or `guery_num`, `sequence_len`,因此我们会先把tensor按`sequence_len`压缩成2D再传入`sequence_mask`中，`sequence_mask`只处理单个`batch_size`的tensor也就是2D tensor）  
下一个参数`valid_lens`是一个列表，里面包含tensor中每一行，也就是每一个单独的key或query的有效长度，超出这个长度的用第三个参数`value`填充  
下面，我们格局具体函数来详细讲解函数的实现细节


In [2]:
def sequence_mask(X, valid_lens, value=0):
    maxlen = X.shape[1]
    mask = torch.arange(maxlen, dtype=torch.float32, device=X.device)[None,:] < valid_lens[:,None]
    X[~mask] = value
    return X

首先，`maxlen = X.shape[1]`用于获取序列长度，因为传入的是一个2dim的tensor，`shape[1]`就表示,这个非常好理解，比较难理解的是下一行代码  
```python
mask = torch.arange(maxlen, dtype=torch.float32, device=X.device)[None,:] < valid_lens[:,None]
```
这行代码非常好的展现了python的易用性和强大的功能。首先要清楚这行代码的最终目的是生成一个全是布尔值的和输入的X大小相同的且在有效长度内值为`True`，有效长度外值为`False`的tensor，为后面填充value做准备。  
为了更好的理解，我们用一个具体的例子来解释如何实现：  
假设X是一个`3x4`的tensor，  
`valide_lens=[1.,2.,3.]`  
则:`maxlen = 4`  
`torch.arange(maxlen, dtype=torch.float32, device=X.device)`这行代码生成了一个maxlen长度的序列，数据格式是`torch.float32`，为了保证数据和X在同一个decive上方便后续的计算加上`device=X.device`。  
因此我们会得到这样一个tensor：`[0.,1.,2.,3.]`  
而后面的`[None,:]`用来增加维度，None在哪个维度上就在哪个位置增加一个维度，原先我们的tensor是1 dim的，`[None,:]`就表示在第0维的位置增加一个维度，于是我们的tensor变成了`1x4`的一个矩阵`[[0.,1.,2.,3.]]`  
同样的，`<`后面`valid_lens[:,None]`就表示在valid_lens的第1维增加一个维度，于是`valid_lens`就变成了一个`3x1`矩阵`[[1.],[2.],[3.]]`这样我们再来比较大小  
那么问题来了，一个`1x3`的一个矩阵要如何和一个`3x1`矩阵比较大小呢？这就要提到pytorch一个**广播（broadcast）**机制，具体规则如下：  
1. 如果两个张量维度数量不同，在维度较少的张量前面补1，直到维度相同
   * 对于我们的两个张量矩阵都是2 dim已经满足这一条件，跳过
2. 从后往前逐个比较两个张量维度大小：
   1. 若维度大小相同，则该维度保持不变
   2. 如果其中一个张量的维度大小是1，将其复制扩展以匹配另一个张量的维度大小
   3. 如果维度大小不同且没有一个是1，则会报错

我们的两个张量矩阵就很好的利用了这一特性对这两个tensor从左到右看，经过广播后都变成3x3的矩阵：
```
[[0.,1.,2.,3.],
 [0.,1.,2.,3.],
 [0.,1.,2.,3.]]
```
```
valid_lens=[[1.,1.,1.,1.],
            [2.,2.,2.,2.],
            [3.,3.,3.,3.]]
```
这样进行比较，得到最终的mask：
```
mask = [[True,False,False,False],
        [True,True, False,False],
        [True,True, True, False]]
```
最后一行`X[~mask] = value`也很巧妙：  
`~mask`将mask反转（True变False，False变True），`X[~mask]`将X在mask中位置维True的值改成value，从而最终实现mask的功能。

下面，masked_softmax函数实现，将我们最初的3 dim的X转化为2 dim的tensor，对输入valid_lens进行必要处理后输入sequence_mask函数，最终将X还原为3 dim后经过softmax函数处理再输出

In [ ]:
def masked_softmax(X, valid_lens):
    if valid_lens is None:
        return nn.functional.softmax(X, dim = -1)
            #如果没有指定valid_lens即无需进行mask，就直接对最后一个维度进行softmax后输出
    else:
        shape = X.shape
            #记录X的形状，获取序列长度和后面还原需要
        if valid_lens.dim() == 1:
            valid_lens = torch.repeat_interleave(valid_lens, shape[1])
                #如果valid_lens是一维的，则说明是按照每一个bach_size来指定有效长度，即每一个batch_size内的有效长度都相同，因此，用repeat_interleave(valid_lens, shape[1])将长度复制扩展和每一行对应，保证输入sequence_mask的valid_lens格式正确
        else: 
            valid_lens = valid_lens.reshape(-1)
                #如果valid_lens不是一维，则说明已经按照行标注好，只需reshape成一维tensor即可
        X = sequence_mask(X.reshape(-1,shape[-1]), valid_lens, -1e6)
    return nn.functional.softmax(X.reshape(shape),dim=-1)

In [7]:
masked_softmax(torch.rand(2, 2, 4), torch.tensor([2, 3]))

tensor([[[0.4497, 0.5503, 0.0000, 0.0000],
         [0.4146, 0.5854, 0.0000, 0.0000]],

        [[0.2204, 0.3066, 0.4730, 0.0000],
         [0.3412, 0.3344, 0.3244, 0.0000]]])

In [8]:
masked_softmax(torch.rand(2, 2, 4), torch.tensor([[1, 3], [2, 4]]))

tensor([[[1.0000, 0.0000, 0.0000, 0.0000],
         [0.3948, 0.3773, 0.2280, 0.0000]],

        [[0.5193, 0.4807, 0.0000, 0.0000],
         [0.2055, 0.2616, 0.3542, 0.1788]]])

### AdditiveAttention


In [ ]:
class AdditiveAttention(nn.Module):
    def __init__(self, key_size, query_size, num_hiddens, dropout):
        super(AdditiveAttention, self).__init__(**kwargs)
        self.W_k = nn.Linear(key_size, num_hiddens, bias=False)
        self.W_q = nn.Linear(query_size, num_hiddens, bias = False)
        self.w_v = nn.Linear(num_hiddens, 1, bias=False)
        self.dropout = nn.Dropout(dropout)
    def forward(self, queries, keys, values, valid_lens):
        queries, keys = self.W_q(queries), self.W_k(keys)
        features = queries.unsqueeze(2) + keys.unsqueeze(1)
        features = torch.tanh(features)
        scores = self.w_v(features).squeeze(-1)
        self.attention_weights = masked_softmax(scores, valid_lens)
        return torch.bmm(self.dropout(self.attention_weights), values)

In [ ]:
class DotProductAttention(nn.Module):
    def __init__(self, dropout, **kwargs):
        super(DotProductAttention, self).__init__( **kwargs)
        self.dropout = nn.Dropout(dropout)

    def forward(self, queries, keys, values, valid_lens=None):
        d = queries.shape[-1]
        scores = torch.bmm(queries, keys.transpose(1, 2)) / math.sqrt(d)
        self.attention_weights = masked_softmax(scores, valid_lens)
        return torch.bmm(self.dropout(self.attention_weights), values)